# Load and inspect the Dataset

In [31]:
import pandas as pd

# Data URL
DATA_URL = "https://raw.githubusercontent.com/Sven-Bo/datasets/master/store_sales_2022-2023.csv"

# Load Dataset to pandas
data = pd.read_csv(DATA_URL)

# Display the data
data.head()

,order_id,product_id,store_id,product_name,product_category,city,date_of_sale,quantity_sold,sales_amount
0,1,52,1,CodeComet,Software Development Tools,Tokyo,1/1/2022,8,303.29
1,2,83,3,SyntaxScribe,Software Development Tools,Yokohama,1/1/2022,8,173.53
2,3,24,3,CodeCanvas,Software Development Tools,Yokohama,1/2/2022,6,37.72
3,4,88,2,VarVista Pro,Educational Tools,Osaka,1/2/2022,6,10.47
4,5,60,1,LoopLantern,Creative & Design Tools,Tokyo,1/3/2022,1,159.10


## Check data type

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          1730 non-null   int64  
 1   product_id        1730 non-null   int64  
 2   store_id          1730 non-null   int64  
 3   product_name      1730 non-null   object 
 4   product_category  1730 non-null   object 
 5   city              1730 non-null   object 
 6   date_of_sale      1730 non-null   object 
 7   quantity_sold     1730 non-null   int64  
 8   sales_amount      1730 non-null   float64
dtypes: float64(1), int64(4), object(4)
memory usage: 121.8+ KB


# Transform Data for Datetime and Extract features

In [33]:
# Convert 'data_of_sale' to datetime
data['date_of_sale'] = pd.to_datetime(data['date_of_sale'])

# Check data type
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          1730 non-null   int64         
 1   product_id        1730 non-null   int64         
 2   store_id          1730 non-null   int64         
 3   product_name      1730 non-null   object        
 4   product_category  1730 non-null   object        
 5   city              1730 non-null   object        
 6   date_of_sale      1730 non-null   datetime64[ns]
 7   quantity_sold     1730 non-null   int64         
 8   sales_amount      1730 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(3)
memory usage: 121.8+ KB


## Extract month and year

In [34]:
data['month'] = data['date_of_sale'].dt.month
data['year'] = data['date_of_sale'].dt.year

# Check the extracted data
data.head()

,order_id,product_id,store_id,product_name,product_category,city,date_of_sale,quantity_sold,sales_amount,month,year
0,1,52,1,CodeComet,Software Development Tools,Tokyo,2022-01-01,8,303.29,1,2022
1,2,83,3,SyntaxScribe,Software Development Tools,Yokohama,2022-01-01,8,173.53,1,2022
2,3,24,3,CodeCanvas,Software Development Tools,Yokohama,2022-01-02,6,37.72,1,2022
3,4,88,2,VarVista Pro,Educational Tools,Osaka,2022-01-02,6,10.47,1,2022
4,5,60,1,LoopLantern,Creative & Design Tools,Tokyo,2022-01-03,1,159.10,1,2022


# Chaining Transformations

In [39]:
# Perform the same transformation using chaining method
data = data.assign(
    date_of_sale=lambda df: pd.to_datetime(df['date_of_sale']),
    month=lambda df: df['date_of_sale'].dt.month,
    year=lambda df: df['date_of_sale'].dt.year
)

# Check the transformed DataFrame
data.head()

,order_id,product_id,store_id,product_name,product_category,city,date_of_sale,quantity_sold,sales_amount,month,year
0,1,52,1,CodeComet,Software Development Tools,Tokyo,2022-01-01,8,303.29,1,2022
1,2,83,3,SyntaxScribe,Software Development Tools,Yokohama,2022-01-01,8,173.53,1,2022
2,3,24,3,CodeCanvas,Software Development Tools,Yokohama,2022-01-02,6,37.72,1,2022
3,4,88,2,VarVista Pro,Educational Tools,Osaka,2022-01-02,6,10.47,1,2022
4,5,60,1,LoopLantern,Creative & Design Tools,Tokyo,2022-01-03,1,159.10,1,2022


# Calculating City Revenues

In [41]:
YEAR = 2023

# Calculate total revenue for each city and year then calculate percentage change
city_revenue = (
    data.groupby(['city', 'quantity_sold', 'year'])['sales_amount']
    .sum()
    .unstack()
    .assign(change=lambda x: x.pct_change(axis=1)[YEAR] *100)
)

city_revenue

year                        2022      2023     change
city     quantity_sold                               
Osaka    1              10058.96   6551.08 -34.873188
         2              10271.79  11204.33   9.078651
         3               8965.55   9230.49   2.955089
         4               7058.00   8349.93  18.304477
         5               8020.99   9004.97  12.267563
         6               6440.88   8853.47  37.457459
         7               8350.73   7611.30  -8.854675
         8              10412.41   8376.30 -19.554647
         9               7335.61  12021.06  63.872670
Tokyo    1               8679.58   5932.84 -31.646001
         2               8935.26   6195.60 -30.661223
         3               7928.88   6907.12 -12.886562
         4               7745.70  12368.95  59.687956
         5              10091.56   6993.92 -30.695353
         6              11656.68   8219.62 -29.485754
         7               6455.31   8946.23  38.587148
         8               9516.11   8412.57 -11.596545
         9               8952.05   8740.81  -2.359683
Yokohama 1               5956.54   8182.61  37.371864
         2               6211.52   7786.64  25.358044
         3               8069.56   6787.54 -15.887112
         4               8800.09   9339.20   6.126187
         5               7081.83   6686.20  -5.586550
         6               6109.22   7254.55  18.747565
         7               6075.11   8044.02  32.409454
         8               6273.78   6550.16   4.405319
         9               8639.25   6479.97 -24.993836